In [1]:
llm_model = 'llama3.2:latest' 
# llm_model =  'deepseek-r1:7b'

In [2]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [3]:
llm = ChatOllama(model=llm_model, temperature=0.9)

In [4]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

In [5]:
chain = LLMChain(llm=llm, prompt=prompt)

/var/folders/0t/xlg3nq8x63j3nxrs9x50yw9m0000gn/T/ipykernel_21358/1305865249.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [6]:
product = "Queen Size Sheet Set"
chain.invoke(product)

{'product': 'Queen Size Sheet Set',
 'text': "Here are some suggestions for a company name that makes Queen-size sheet sets:\n\n1. **Royal Sheets**: A nod to the iconic British monarch, conveying luxury and comfort.\n2. **Queen's Bedding Co.**: Straightforward and regal, emphasizing the focus on queen-size sheet sets.\n3. **Sheets of Majesty**: Evoking feelings of grandeur and opulence, perfect for a premium bedding brand.\n4. **Regalia Linens**: Suggesting elegance and sophistication, with a touch of royalty.\n5. **The Royal Blanket Co.**: Emphasizing the comfort and warmth of a high-quality queen-size sheet set.\n6. **Queen's Rest Bedding**: Conveying a sense of relaxation and rejuvenation, perfect for those seeking a good night's sleep.\n7. **Luxe Sheets by [Your Name]**: Adding a personal touch, with a focus on high-end quality and comfort.\n8. **The Queen's Suite**: Suggesting an all-encompassing bedding solution that meets the needs of even the most discerning customers.\n\nChoos

SimpleSequentialChain

In [7]:
from langchain.chains import SimpleSequentialChain

In [8]:
# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [9]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [10]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [11]:
overall_simple_chain.run(product)

/var/folders/0t/xlg3nq8x63j3nxrs9x50yw9m0000gn/T/ipykernel_21358/228193363.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_simple_chain.run(product)




> Entering new SimpleSequentialChain chain...
Here are some suggestions for a company name that sells queen-size sheet sets:

1. **Royal Sheets**: A straightforward name that evokes the idea of luxury and royalty, which fits well with the queen-size theme.
2. **Queen's Rest**: This name plays on the idea of getting a good night's sleep, which is what customers are looking for when they buy sheets.
3. **Sheet Sanctuary**: This name suggests a safe haven where customers can find comfortable and cozy bedding.
4. **Regal Linens**: "Regal" conveys a sense of luxury and high-end quality, while "linens" is a general term that encompasses the company's products.
5. **Queen's Comfort Co.**: Simple and straightforward, this name tells customers exactly what to expect from their sheets.
6. **Soft Touch Bedding**: This name emphasizes the comfort and softness of the sheets, which is likely a key selling point for customers.
7. **Her Majesty's Sheets**: A playful and whimsical name that adds a to

'Here are the descriptions for each of the company names:\n\n1. **Royal Sheets**: "Luxurious queen-size sheet sets fit for royalty."\n\n2. **Queen\'s Rest**: "Find peaceful slumber with our high-quality queen-size sheets."\n\n3. **Sheet Sanctuary**: "Cozy up in comfort with our soothing queen-size bedding solutions."\n\n4. **Regal Linens**: "Experience the best of luxury with our regal and refined queen-size linens."\n\n5. **Queen\'s Comfort Co.**: "Your trusted source for soft, comfortable queen-size sheets."\n\n6. **Soft Touch Bedding**: "Indulge in the comfort of our luxurious queen-size bedding."\n\n7. **Her Majesty\'s Sheets**: "Whimsical queen-size sheet sets fit for a royal night\'s sleep."\n\n8. **Bedazzled Bedding Co.**: "Elevate your sleep with exceptional, bedazzling queen-size bedding."\n\n9. **The Queen\'s Closet**: "Treasured queen-size sheets for the discerning sleeper."\n\n10. **Slumber & Luxe**: "Surrender to restful slumber and luxurious comfort in every thread."'

Sequential Chain

In [12]:
from langchain.chains import SequentialChain

In [13]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [14]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key="summary"
                    )


In [15]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key="language"
                      )


In [16]:

# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )


In [17]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [18]:
review = "J'ai acheté ce produit pour mon fils et il l'adore. Il est très confortable et doux. Le seul problème est qu'il est un peu petit pour lui."
overall_chain(review)



> Entering new SequentialChain chain...


/var/folders/0t/xlg3nq8x63j3nxrs9x50yw9m0000gn/T/ipykernel_21358/3303964796.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  overall_chain(review)



> Finished chain.


{'Review': "J'ai acheté ce produit pour mon fils et il l'adore. Il est très confortable et doux. Le seul problème est qu'il est un peu petit pour lui.",
 'English_Review': 'Here is the translation of the review:\n\n"I bought this product for my son and he loves it. It\'s very comfortable and soft. The only issue is that it\'s a bit small for him."',
 'summary': 'The reviewer purchased this product for their son, who loves its comfort and softness, but finds it too small for his needs.',
 'followup_message': "Réponse de suivi :\n\nMalheureusement, malgré son enthousiasme pour les confort et la douceur du produit, mon fils se sent toujours un peu étroit dessus. Il aimerait une taille plus grande pour pouvoir s'adapter confortablement à ses besoins quotidiens. Malgré cela, il continue à l'utiliser régulièrement car il aime son style et sa qualité, mais j'aurais aimé qu'il fût disponible en taille plus grande."}

Router Chain

In [19]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [20]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [21]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [22]:
llm = ChatOllama(model=llm_model, temperature=0.5)

In [23]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [24]:
print(destinations)

['physics: Good for answering questions about physics', 'math: Good for answering math questions', 'History: Good for answering history questions', 'computer science: Good for answering computer science questions']


In [25]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [26]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ "DEFAULT" or name of the prompt to use in {destinations}
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: The value of “destination” MUST match one of \
the candidate prompts listed below.\
If “destination” does not fit any of the specified prompts, set it to “DEFAULT.”
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

<>:1: SyntaxWarning: invalid escape sequence '\ '
<>:1: SyntaxWarning: invalid escape sequence '\ '
/var/folders/0t/xlg3nq8x63j3nxrs9x50yw9m0000gn/T/ipykernel_21358/3710404048.py:1: SyntaxWarning: invalid escape sequence '\ '
  MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \


In [27]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [28]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

/var/folders/0t/xlg3nq8x63j3nxrs9x50yw9m0000gn/T/ipykernel_21358/3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [29]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What are the characteristics of black body radiation in terms of its spectral distribution and temperature dependence?'}
> Finished chain.


"Black-body radiation! One of the most fascinating topics in physics.\n\nThe spectral distribution of black-body radiation, also known as Planck's law, describes how the energy is distributed across different wavelengths. The key characteristics are:\n\n1. **Spectral lines**: Black-body radiation emits a continuous spectrum of light, but it has distinct peaks at specific wavelengths, which correspond to the energy levels of the photons.\n2. **Temperature dependence**: The spectral distribution changes with temperature. As the temperature increases, the peak wavelength shifts towards shorter wavelengths (bluer colors), and the intensity of the emission increases.\n\nThe relationship between temperature and wavelength is described by Wien's displacement law: Δλ = B / T, where Δλ is the change in wavelength, B is a constant, and T is the absolute temperature.\n\nIn terms of spectral distribution, black-body radiation can be divided into several regions:\n\n* **Infrared (IR)**: Temperature

In [30]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'A simple yet fundamental question!\n\nTo break it down, let\'s consider what we\'re trying to find: the result of adding 2 and 2 together.\n\nThe first step is to recall the basic properties of addition. In this case, we can use the commutative property, which states that the order in which we add two numbers doesn\'t change the result. So, we know that:\n\na + b = b + a\n\nIn our case, a = 2 and b = 2.\n\nNext, we can apply the associative property of addition, which allows us to regroup the terms when adding multiple numbers together. However, in this simple case, it\'s not necessary.\n\nNow, let\'s look at the actual numbers involved: 2 + 2. We can think of this as "how many units do we have altogether?" \n\nSince both numbers are equal (both are 2), we\'re essentially counting how many times we have 2 added to itself. This is equivalent to multiplying 2 by 1, since any number multiplied by 1 remains the same.\n\nSo, using basic arithmetic operations, we can rewrite this as:\n\n2 +

In [31]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
physics: {'input': 'Why does the double helix structure of DNA have specific physical properties that allow it to store genetic information?'}
> Finished chain.


"The double helix structure of DNA is indeed a remarkable molecule, and its unique physical properties play a crucial role in storing genetic information. The main reason for this is due to the combination of hydrogen bonding, base pairing, and the geometry of the molecule.\n\nIn the double helix structure, two complementary strands of nucleotides are twisted together, with each nucleotide consisting of a sugar molecule (deoxyribose), a phosphate group, and one of four nitrogenous bases: adenine (A), guanine (G), cytosine (C), and thymine (T).\n\nThe hydrogen bonding between the nitrogenous bases is what holds the double helix structure together. The base pairing rules state that:\n\n* Adenine (A) pairs with Thymine (T)\n* Guanine (G) pairs with Cytosine (C)\n\nThese hydrogen bonds are weak, but they provide a stable framework for the molecule. The geometry of the molecule also plays a crucial role, as the sugar-phosphate backbone provides a rigid structure that allows the double helix